# Chapter 14 - Excel Spreadsheets

## Notes

### A REVIEW OF WORKBOOKS, SHEETS, AND CELLS

As a quick review, here’s a rundown of all the functions, methods, and data types involved in reading a cell out of a spreadsheet file:

1.  Import the openpyxl module.

2.  Call the openpyxl.load_workbook() function to get a Workbook object.

3.  Use the active or sheetnames attribute.

4.  Get a Worksheet object.

5.  Use indexing or the cell() sheet method with row and column keyword arguments.

6.  Get a Cell object.

7.  Read the Cell object’s value attribute.

### Project 9: Gather Census Statistics

Say you have a spreadsheet of data from the 2010 US Census and you’ve been given the boring task of going through its thousands of rows to count both the population and the number of census tracts for each county. (A census tract is simply a geographic area defined for the purposes of the census.) Each row represents a single census tract. We’ll name the spreadsheet file censuspopdata.xlsx, and you can download it from this book’s online resources. Its contents look like Figure 14-2.

Even though Excel can automatically calculate the sum of multiple selected cells, you’d still have to first manually select the cells for each of the 3,000-plus counties. Even if it takes just a few seconds to calculate a county’s population by hand, this would take hours to do for the whole spreadsheet.

In this project, you’ll write a script that can read from the census spreadsheet file and calculate statistics for each county in a matter of seconds.

This is what your program does:

- Reads the data from the Excel spreadsheet  
- Counts the number of census tracts in each county  
- Counts the total population of each county  
- Prints the results  

This means your code will need to do the following:

- Open and read the cells of an Excel document with the openpyxl module.
- Calculate all the tract and population data and store it in a data structure.
- Write the data structure to a text file with the .py extension using the pprint module so that it can be imported later.

#### Step 1: Read the Spreadsheet Data
There is just one sheet in the censuspopdata.xlsx spreadsheet, named 'Population by Census Tract', and each row in the sheet holds the data for a single census tract. The columns are the tract number (A), the state abbreviation (B), the county name (C), and the population of the tract (D).

Open a new file editor tab and enter the following code, then save the file as readCensusExcel.py:
```
# readCensusExcel.py - Tabulates county population and census tracts

❶ import openpyxl, pprint
print('Opening workbook...')
❷ wb = openpyxl.load_workbook('censuspopdata.xlsx')
❸ sheet = wb['Population by Census Tract']
county_data = {}

# TODO: Fill in county_data with each county's population and tracts.
print('Reading rows...')
❹ for row in range(2, sheet.max_row + 1):
    # Each row in the spreadsheet has data for one census tract.
    state  = sheet['B' + str(row)].value
    county = sheet['C' + str(row)].value
    pop    = sheet['D' + str(row)].value

# TODO: Open a new text file and write the contents of county_data to it.
```
This code imports the openpyxl module, as well as the pprint module that you’ll use to print the final county data ❶. Then, it opens the censuspopdata.xlsx file ❷, gets the sheet with the census data ❸, and begins iterating over its rows ❹.

Note that you’ve also created a variable named county_data, which will contain the populations and number of tracts you calculate for each county. Before you can store anything in it, though, you should determine exactly how you’ll structure the data inside it.

#### Step 2: Populate the Data Structure
In the United States, states have two-letter abbreviations and are further split into counties. The data structure stored in county_data will be a dictionary with state abbreviations as its keys. Each state abbreviation will map to another dictionary, whose keys are strings of the county names in that state. Each county name will in turn map to a dictionary with just two keys, 'tracts' and 'pop'. These keys map to the number of census tracts and the population for the county. For example, the dictionary will look similar to this:

{'AK': {'Aleutians East': {'pop': 3141, 'tracts': 1},
        'Aleutians West': {'pop': 5561, 'tracts': 2},
        'Anchorage': {'pop': 291826, 'tracts': 55},
        'Bethel': {'pop': 17013, 'tracts': 3},
        'Bristol Bay': {'pop': 997, 'tracts': 1},
        --snip--
If the previous dictionary were stored in county_data, the following expressions would evaluate like this:
```
>>> county_data['AK']['Anchorage']['pop']
291826
>>> county_data['AK']['Anchorage']['tracts']
55
More generally, the county_data dictionary’s keys will look like this:

county_data[state abbrev][county]['tracts']
county_data[state abbrev][county]['pop']
Now that you know how county_data will be structured, you can write the code that will fill it with the county data. Add the following code to the bottom of your program:

# readCensusExcel.py - Tabulates county population and census tracts

--snip--

for row in range(2, sheet.max_row + 1):
    # Each row in the spreadsheet has data for one census tract.
    state  = sheet['B' + str(row)].value
    county = sheet['C' + str(row)].value
    pop    = sheet['D' + str(row)].value

    # Make sure the key for this state exists.
  ❶ county_data.setdefault(state, {})
    # Make sure the key for this county in this state exists.
  ❷ county_data[state].setdefault(county, {'tracts': 0, 'pop': 0})

    # Each row represents one census tract, so increment by one.
  ❸ county_data[state][county]['tracts'] += 1
    # Increase the county pop by the pop in this census tract.
  ❹ county_data[state][county]['pop'] += int(pop)

# TODO: Open a new text file and write the contents of county_data to it.
```
The last two lines of code perform the actual calculation work, incrementing the value for tracts ❸ and increasing the value for pop ❹ for the current county on each iteration of the for loop.

The other code is there because you cannot add a county dictionary as the value for a state abbreviation key until the key itself exists in county_data. (That is, county_data['AK']['Anchorage']['tracts'] += 1 will cause an error if the 'AK' key doesn’t exist yet.) To make sure the state abbreviation key exists in your data structure, you need to call the setdefault() method to set a value if one does not already exist for state ❶.

Just as the county_data dictionary needs a dictionary as the value for each state abbreviation key, each of those dictionaries will need its own dictionary as the value for each county key ❷. And each of those dictionaries in turn will need the keys 'tracts' and 'pop' that start with the integer value 0. (If you ever lose track of the dictionary structure, look back at the example dictionary at the start of this section.)

Since setdefault() will do nothing if the key already exists, you can call it on every iteration of the for loop without a problem.

#### Step 3: Write the Results to a File
After the for loop has finished, the county_data dictionary will contain all of the population and tract information keyed by county and state. At this point, you could program more code to write this data to a text file or another Excel spreadsheet. For now, let’s just use the pprint.pformat() function to write the county_data dictionary value as a massive string to a file named census2010.py. Add the following code to the bottom of your program (making sure to keep it un-indented so that it stays outside the for loop):
```
# readCensusExcel.py - Tabulates county population and census tracts.

--snip--

# Open a new text file and write the contents of county_data to it.
print('Writing results...')
result_file = open('census2010.py', 'w')
result_file.write('allData = ' + pprint.pformat(county_data))
result_file.close()
print('Done.')
The pprint.pformat() function produces a string that itself is formatted as valid Python code. By outputting it to a text file named census2010.py, you’ve generated a Python program from your Python program! This may seem complicated, but the advantage is that you can now import census2010.py just like any other Python module. In the interactive shell, change the current working directory to the folder with your newly created census2010.py file and then import it:

>>> import census2010
>>> census2010.allData['AK']['Anchorage']
{'pop': 291826, 'tracts': 55}
>>> anchorage_pop = census2010.allData['AK']['Anchorage']['pop']
>>> print('The 2010 population of Anchorage was ' + str(anchorage_pop))
The 2010 population of Anchorage was 291826
```
The readCensusExcel.py program was throwaway code: once you have its results saved to census2010.py, you won’t need to run the program again. Whenever you need the county data, you can just run import census2010.

Calculating this data by hand would have taken hours; this program did it in a few seconds. Using openpyxl, you’ll have no trouble extracting information saved to an Excel spreadsheet and performing calculations on it. You can download the complete program from the book’s online resources.

### Project 10: Update a Spreadsheet

In this project, you’ll write a program to update cells in a spreadsheet of produce sales. Your program will look through the spreadsheet, find specific kinds of produce, and update their prices. Download this produceSales3.xlsx spreadsheet from the book’s online resources. Figure 14-3 shows what the spreadsheet looks like.

Each row represents an individual sale. The columns are the type of produce sold (A), the cost per pound of that produce (B), the number of pounds sold (C), and the total revenue from the sale (D). The TOTAL column is set to an Excel formula like =ROUND(B2*C2, 2), which multiplies the row’s cost per pound by the number of pounds sold and rounds the result to the nearest cent. With this formula, the cells in the TOTAL column will automatically update themselves if there is a change in the COST PER POUND and POUNDS SOLD columns.

Now imagine that the prices of garlic, celery, and lemons were entered incorrectly, leaving you with the boring task of going through thousands of rows in this spreadsheet to update the cost per pound for any celery, garlic, and lemon rows. You can’t do a simple find-and-replace for the price, because there might be other items with the same price that you don’t want to mistakenly “correct.” For thousands of rows, this would take hours to do by hand. But you can write a program that can accomplish this in seconds.

Your program should do the following:

- Loop over all the rows.  
- If the row is for celery, garlic, or lemons, change the price.  

This means your code will need to do the following:

- Open the spreadsheet file.  
- For each row, check whether the value in column A is Celery, Garlic, or Lemon.  
- If it is, update the price in column B.  
- Save the spreadsheet to a new file (so that you don’t lose the original spreadsheet, just in case).

#### Step 1: Set Up a Data Structure with the Updated Information
The prices that you need to update are as follows:
```
Celery: 1.19
Garlic: 3.07
Lemon: 1.27
You could write code to set these new prices, like this:

if produce_name == 'Celery':
    cell_obj = 1.19
if produce_name == 'Garlic':
    cell_obj = 3.07
if produce_name == 'Lemon':
    cell_obj = 1.27
```

But hardcoding the produce and updated price data like this is a bit inelegant. If you needed to update the spreadsheet again with different prices or different produce, you would have to change a lot of the code. Every time you change code, you risk introducing bugs.

A more flexible solution is to store the corrected price information in a dictionary and write your code to use this data structure. In a new file editor tab, enter the following code:
```
# updateProduce.py - Corrects costs in produce sales spreadsheet

import openpyxl

wb = openpyxl.load_workbook('produceSales3.xlsx')
sheet = wb['Sheet']

# The produce types and their updated prices
PRICE_UPDATES = {'Garlic': 3.07,
                 'Celery': 1.19,
                 'Lemon': 1.27}

# TODO: Loop through the rows and update the prices.
```
Save this as updateProduce.py. If you need to update the spreadsheet again, you’ll need to update only the PRICE_UPDATES dictionary, not any other code.

#### Step 2: Check All Rows and Update Incorrect Prices
The next part of the program will loop through all the rows in the spreadsheet. Add the following code to the bottom of updateProduce.py:
```
# updateProduce.py - Corrects costs in produce sales spreadsheet

--snip--

# Loop through the rows and update the prices.
❶ for row_num in range(2, sheet.max_row + 1):  # Skip the first row.
  ❷ produce_name = sheet.cell(row=row_num, column=1).value
  ❸ if produce_name in PRICE_UPDATES:
        sheet.cell(row=row_num, column=2).value = PRICE_UPDATES[produce_name]

❹ wb.save('updatedProduceSales3.xlsx')
```
We loop through the rows starting at row 2, as row 1 is just the header ❶. The cell in column 1 (that is, column A) will be stored in the variable produce _name ❷. If produce_name exists as a key in the PRICE_UPDATES dictionary ❸, you know this row needs its price corrected. The correct price will be in PRICE _UPDATES[produce_name].

Notice how clean using PRICE_UPDATES makes the code. It uses only one if statement, rather than a separate line like if produce_name == 'Garlic': for every type of produce to update. And since the code uses the PRICE_UPDATES dictionary instead of hardcoding the produce names and updated costs into the for loop, you can modify only the PRICE_UPDATES dictionary, and not the rest of the code, if the produce sales spreadsheet needs additional changes.

After going through the entire spreadsheet and making changes, the code saves the Workbook object to updatedProduceSales3.xlsx ❹. It doesn’t overwrite the old spreadsheet, in case there’s a bug in the program and the updated spreadsheet is wrong. After checking that the updated spreadsheet looks right, you can delete the old spreadsheet.

## Practice Questions

1. What does the openpyxl.load_workbook() function return?  
    **Answer:** Returns a Workbook object.

2. What does the wb.sheetnames workbook attribute contain?  
    **Answer:** List of all sheetnames in the excel workbook.

3. How would you retrieve the Worksheet object for a sheet named 'Sheet1'?  
    **Answer:** `sheet = wb['Sheet1']`

4. How would you retrieve the Worksheet object for the workbook’s active sheet?  
    **Answer:** `wb.active`

5. How would you retrieve the value in cell C5?  
    **Answer:** `sheet['C5'].value`

6. How would you set the value in cell C5 to "Hello"?  
    **Answer:** `sheet['C5'] = "Hello"`

7. How would you retrieve the cell’s row and column as integers?  
    **Answer:**  
    ```
    c = sheet['A1']
    r, c = c.row, c.column
    ```

8. What do the sheet.max_column and sheet.max_row sheet attributes hold, and what is the data type of these attributes?  
    **Answer:** Gets the highest column and row numbers respectively.

9. If you needed to get the integer index for column 'M', what function would you need to call?  
    **Answer:**
    ```
    from openpyxl.utils import column_index_from_string


    column_index_from_string('M')
    ```

10. If you needed to get the string name for row 14, what function would you need to call?  
    **Answer:** `openpyxl.utils.get_column_letter(14)`

11. How can you retrieve a tuple of all the Cell objects from A1 to F1?  
    **Answer:** `sheet['A1':'F1']`


12. How would you save the workbook to the filename example3.xlsx?  
    **Answer:** `wb.save('example3.xlsx')`

13. How do you set a formula in a cell?  
    **Answer:** `sheet['B9'] = '=SUM(B1:B8)'`

14. If you want to retrieve the result of a cell’s formula instead of the cell’s formula itself, what must you do first?  
    **Answer:**
    ```
    wb = openpyxl.load_workbook('writeFormula3.xlsx', data_only=True)  # Open with data_only.
    wb.active['A3'].value
    ```

15. How would you set the height of row 5 to 100?  
    **Answer:** `sheet.row_dimensions[1].height = 100`

16. How would you hide column C?  
    **Answer:** `sheet.column_dimensions['C'].hidden = True`

17. What is a freeze pane?  
    **Answer:** For spreadsheets that are too large to be displayed all at once, it’s helpful to “freeze” a few of the top rows or leftmost columns onscreen. Frozen column or row headers, for example, are always visible to the user even as they scroll through the spreadsheet. These are known as *freeze panes*.

18. What five functions and methods do you have to call to create a bar chart?  
    **Answer:**
    ```
    import openpyxl
    wb = openpyxl.Workbook()
    sheet = wb.active
    for i in range(1, 11):  # Create some data in column A.
        sheet['A' + str(i)] = i * i
        
    ref_obj = openpyxl.chart.Reference(sheet, 1, 1, 1, 10)

    series_obj = openpyxl.chart.Series(ref_obj, title='First series')

    chart_obj = openpyxl.chart.BarChart()
    chart_obj.title = 'My Chart'
    chart_obj.append(series_obj)

    sheet.add_chart(chart_obj, 'C5')
    wb.save('sampleChart3.xlsx')
    ```

## Practice Programs

For practice, write programs to do the following tasks.

### Multiplication Table Maker
Create a program multiplicationTable.py that takes a number N from the command line and creates an N×N multiplication table in an Excel spreadsheet. For example, when the program is run like this:

`py multiplicationTable.py 6`  
it should create a spreadsheet that looks like Figure 14-10. Row 1 and column A should contain labels and be in bold.

**"multiplication_table.py"**

### Blank Row Inserter
Create a program blankRowInserter.py that takes two integers and a filename string as command line arguments. Let’s call the first integer N and the second integer M. Starting at row N, the program should insert M blank rows into the spreadsheet. For example, when the program is run like this:

`python blankRowInserter.py 3 2 myProduce.xlsx`  
the “before” and “after” spreadsheets should look like Figure 14-11. You can write this program by reading in the contents of the spreadsheet. Then, when writing out the new spreadsheet, use a for loop to copy the first N lines. For the remaining lines, add M to the row number in the output spreadsheet.

**"insert_rows.py"**